In [ ]:
import torch
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os


class ExpDataset(torch.utils.data.Dataset):
    def __init__(self, c1, l1, c2, l2):
        super().__init__()
        self.l1 = l1
        self.l2 = l2
        self.c1 = c1
        self.c2 = c2

    def __len__(self):
        return len(self.l1)

    def __getitem__(self, idx):
        return self.c1[idx], self.l1[idx], self.c2[idx], self.l2[idx]


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.k = torch.nn.Parameter(torch.tensor(1., requires_grad=True, dtype=torch.float64), requires_grad=True)

    def forward(self):
        ret = (self.k + 1) / (self.k * 5 + 3)
        return ret


def train(data: pd.DataFrame):
    num_epochs = 8000

    net = Net()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    c1_data = torch.tensor(data['c1'].values, dtype=torch.float64).view(-1, 1)
    l1_data = torch.tensor(data['l1'].values, dtype=torch.float64).view(-1, 1)
    c2_data = torch.tensor(data['c2'].values, dtype=torch.float64).view(-1, 1)
    l2_data = torch.tensor(data['l2'].values, dtype=torch.float64).view(-1, 1)
    dataset = ExpDataset(c1_data, l1_data, c2_data, l2_data)
    loader = torch.utils.data.DataLoader(dataset, batch_size=6, shuffle=True)

    for epoch in range(num_epochs):
        for c1, l1, c2, l2 in loader:
            optimizer.zero_grad()
            pred = net()
            loss = criterion(pred, l1 * c1 / l2 / c2)
            loss.backward()
            optimizer.step()
        if epoch % 500 == 0:
            with torch.no_grad():
                pred = net()
                loss = criterion(pred, l1_data * c1_data / l2_data / c2_data)
                print("Epoch", epoch, "loss", loss.item())
    return net


def kregression(data):
    #——编写k的求解程序----#
    #---solve k in this function----#
    net = train(data)
    return net.k.item()

In [ ]:
if __name__ == '__main__':
    #-------------读取训练集,训练集地址已经设定好，不用修改------------------#
    #--------Read the training set, the address of the training set has been set and does not need to be modified------------#
    train_path = "/bohr/train-0ahg/v1/data_train.csv"
    data_train = pd.read_csv(train_path)

    #-------------读取测试集，“DATA_PATH”是测试集加密后的环境变量，按照如下方式可以在提交后，系统评分时访问测试集，但是选手无法直接下载---------#
    # Read the test set, “DATA_PATH” is the environment variable encrypted for the test set. It can be accessed during system scoring after submission as follows, but the participants cannot directly download it.#
    if os.environ.get('DATA_PATH'):
        DATA_PATH = os.environ.get("DATA_PATH") + "/"
    else:
        print("Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象")  #Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象
        print("When baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.")
    test_path = DATA_PATH + "data_test.csv"
    data_test = pd.read_csv(test_path)
    k_train = kregression(data_train)
    k_test = kregression(data_test)
    k = [k_train,k_test]
    print(k_train,k_test)
    df_params = pd.DataFrame({'k': k})
    # 将参数转换为DataFrame
    print(df_params)
    # 保存到CSV文件
    csv_file_path = 'submission.csv'
    df_params.to_csv(csv_file_path, index=False)